In [2]:
import json
import cv2
import uuid
import numpy as np
import LineOCR
from PIL import Image

In [5]:
data_dir = "D:/PPYData/test/"
raw_labeled_data = open(f"{data_dir}Label.txt", "r",  encoding="utf-8").readlines()

In [3]:
selected_key_cls = [
    "hospital_name",
    "department",
    "patient_name",
    "age",
    "gender",
    "admission_date",
    "discharge_date",
    "diagnose",
    "sign_date",
]

In [4]:

lineDetAndOCR = LineOCR.ProcessImage()

In [5]:
img_path, test_json = raw_labeled_data[0].split("\t")
img_path = data_dir.replace("test", img_path)[:-1]
raw_img = cv2.imread(img_path)
djson = json.loads(test_json)

In [6]:
croped_dir = "croped_for_ocr/"

In [7]:
import tqdm

In [11]:
lines = []
for each_file in tqdm.tqdm(raw_labeled_data):
    img_path, test_json = each_file.split("\t")
    img_path = data_dir.replace("test", img_path)[:-1]
    raw_img = cv2.imread(img_path)
    djson = json.loads(test_json)
    for each in djson:
        if each["key_cls"] in selected_key_cls:
            cls = each["key_cls"]
            transcription = each["transcription"]
            test_points = each["points"]
            croped_img, _points = lineDetAndOCR.crop_line_v2(raw_img, test_points, convert_gray=True)
            pil_croped_img = Image.fromarray(croped_img)
            text, prob = lineDetAndOCR.individual_OCR(pil_croped_img)
            img_name = str(uuid.uuid4()) + ".jpg"
            line = f"{img_name}\t{text}\t{prob}"
            lines.append(line)
            save_path = croped_dir + img_name
            cv2.imwrite(save_path ,croped_img)

            # print(f"{cls}: {transcription}")
            # print(f"prediction: {text}")
            # print("----------------------")

100%|██████████| 174/174 [01:29<00:00,  1.95it/s]


In [12]:
fname, text, score = [],[],[]
for each in lines:
    _fname, _text, _score = each.split("\t")
    fname.append(_fname)
    text.append(_text)
    score.append(_score)

In [13]:
import pandas as pd

INFO 2022-10-24 12:30:15,781 utils.py:147] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.


In [14]:
df = pd.DataFrame.from_dict({
    "fname": fname,
    "text": text,
    "score": score,
})

In [15]:
sorted_df = df.sort_values("score")

In [16]:
wlines = []
for idx in range(len(sorted_df)):
    row = sorted_df.iloc[idx]
    _fname = row["fname"]
    _text = row["text"]
    _score = row["score"]
    line = f"{_fname}\t{_text}\t{_score}"
    wlines.append(line)

In [17]:
with open("ocr_prediction.txt", "w", encoding="utf-8") as f:
    f.writelines("\n".join(wlines))

In [6]:
img_path, test_json = raw_labeled_data[5].split("\t")
img_path = data_dir.replace("test", img_path)[:-1]
raw_img = cv2.imread(img_path)
djson = json.loads(test_json)

In [7]:
djson[3]

{'transcription': 'SƠ Y TẾ THANH HÓA',
 'points': [[95, 107], [288, 99], [288, 117], [95, 125]],
 'difficult': False,
 'key_cls': 'None'}

In [8]:
points = djson[3]["points"]
croped, temp = lineDetAndOCR.crop_line_v2(raw_img, points, True)

[[[ 95 107]
  [288  99]
  [288 117]
  [ 95 125]]]
(0, 0, 0)


In [9]:
from PIL import Image

In [9]:
raw_img.shape

(738, 1024, 3)

In [ ]:
polygon = djson[3]["points"]
tuple_polygon = []
for each in polygon:
    tuple_polygon.append(tuple(each))
mask = np.zeros((738, 1024))
myROI = tuple_polygon  # (x, y)
roi_img = cv2.fillPoly(mask, [np.array(myROI)], 255)

In [75]:
cv2.imwrite("test.jpg", roi_img)

True

In [109]:
from turtle import fillcolor
import cv2
import numpy as np

# original image
# -1 loads as-is so if it will be 3 or 4 channel as the original
# image = cv2.imread('image.png', -1)
image = raw_img.copy()
# mask defaulting to black for 3-channel and transparent for 4-channel
# (of course replace corners with yours)

polygon = djson[3]["points"]
list_x = [x for x,y in polygon]
list_y = [y for x,y in polygon]
xmin = min(list_x)
xmax = max(list_x)
ymin = min(list_y)
ymax = max(list_y)
tuple_polygon = []
for each in polygon:
    tuple_polygon.append(tuple(each))
    
mask = np.ones(image.shape, dtype=np.uint8)*255

# roi_corners = np.array([[(10,10), (300,300), (10,300)]], dtype=np.int32)
roi_corners = [tuple_polygon]
roi_corners = np.array(roi_corners)
print(roi_corners)
# fill the ROI so it doesn't get wiped out when the mask is applied
channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
ignore_mask_color = (0,)*channel_count
print(ignore_mask_color)
cv2.fillPoly(mask, roi_corners, ignore_mask_color)
# from Masterfool: use cv2.fillConvexPoly if you know it's convex

# apply the mask
masked_image = cv2.bitwise_or(image, mask)

croped_mask = masked_image[ymin:ymax, xmin:xmax]
# save the result
cv2.imwrite('image_masked.png', croped_mask)

[[[ 95 107]
  [288  99]
  [288 117]
  [ 95 125]]]
(0, 0, 0)


True

In [6]:
import tqdm
for each_file in tqdm.tqdm(raw_labeled_data):
    img_path, test_json = each_file.split("\t")
    img_path = data_dir.replace("test", img_path)[:-1]
    raw_img = cv2.imread(img_path)
    djson = json.loads(test_json)
    have_adm_date = False
    have_dis_date = False
    adm_count = 0
    dis_count = 0
    for each in djson:
        if each["key_cls"]== "admission_date":
            have_adm_date = True
            adm_count+=1
        elif each["key_cls"]== "discharge_date":
            have_dis_date = True
            dis_count+=1
        # if have_adm_date == True and have_dis_date == True:
        #     break
    if have_adm_date == False:
        print(f"file {img_path} is missing ADMISSION DATE")
    if have_dis_date == False:
        print(f"file {img_path} is missing DISCHARGE DATE")
    if adm_count>1:
        print(f"file {img_path} MULTI ADMISSION DATE")
    if dis_count>1:
        print(f"file {img_path} MULTI DISCHARGE DATE")


 32%|███▏      | 73/226 [00:00<00:01, 102.47it/s]

file D:/PPYData/test/60.jpeg is missing ADMISSION DATE
file D:/PPYData/test/60.jpeg is missing DISCHARGE DATE


 58%|█████▊    | 132/226 [00:01<00:01, 82.87it/s]

file D:/PPYData/test/123.jpeg MULTI ADMISSION DATE
file D:/PPYData/test/123.jpeg MULTI DISCHARGE DATE


 86%|████████▋ | 195/226 [00:02<00:00, 64.79it/s]

file D:/PPYData/test/194.jpeg MULTI DISCHARGE DATE


100%|██████████| 226/226 [00:02<00:00, 79.15it/s]


In [14]:
import tqdm
for each_file in tqdm.tqdm(raw_labeled_data):
    img_path, test_json = each_file.split("\t")
    img_path = data_dir.replace("test", img_path)[:-1]
    raw_img = cv2.imread(img_path)
    djson = json.loads(test_json)
    have_age = False
    for each in djson:
        if have_age ==False and each["key_cls"]== "diagnose":
            have_age = True
    if have_age == False:
        print(f"file {img_path} is missing AGE")


 34%|███▍      | 77/226 [00:00<00:01, 106.89it/s]

file D:/PPYData/test/60.jpeg is missing AGE


 88%|████████▊ | 199/226 [00:02<00:00, 83.61it/s]

file D:/PPYData/test/190.jpeg is missing AGE


100%|██████████| 226/226 [00:02<00:00, 92.07it/s]
